# Keywords matching and relevance index ranking 

In [ ]:
# import packages
import pandas as pd
import numpy as np
import math
from nltk import ngrams
from matplotlib import pyplot as plt

In [ ]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import parameters as params
import dataframe_columns as df_col

## Predefined fucntions:

In [ ]:
def relevance_index(counts_dict):
    relevance_index = 0
    for key in counts_dict.keys():
        relevance_index += math.log(1+counts_dict[key]) * (params.ranking_kw_groups_weights[key])
    return relevance_index
# --------------------start of test code--------------------
# keywords_count_or_fre = {}
# index = calcul_related(keywords_count_or_fre, params.on_topic_kws_weights)
# print(index)
# ---------------------end of test code---------------------

In [ ]:
def compute_relevance_index(db_count_path, db_relevance_index_path, rank_by):
    df = pd.read_csv(db_count_path, header=0, sep="\t")
    if rank_by == 'tak':
        column_name = '_COUNT_IN_TAK'
    elif rank_by == '500':
        column_name = '_COUNT_IN_500'
    elif rank_by == 'full_text':
        column_name = '_COUNT_IN_FULL_TEXT'
        
    count_dict = {}
    for ind in df.index:
        for key in params.ranking_kw_groups.keys():
            value = df.at[ind, key+column_name]
            if value != value:
                value = 0
            count_dict[key] = int(value)
        # print(count_dict)

        relev_index = relevance_index(count_dict)
        
        df.at[ind, "RELEVANCE_INDEX"] = relev_index

        line_number_in_csv = ind + 1
        print("Line number:", line_number_in_csv, " INDEX:", int(df.at[ind, "INDEX"]))
    
    df.columns = df_col.db_ranked_columns
    df.reset_index(drop=True, inplace=True)
    df.to_csv(db_relevance_index_path, header=True, index=False, sep="\t")
    
    print("Weighting and ranking the potentially related literature succeded!")
    print("Enjoy reading!")
# --------------------start of test code--------------------
# input_path = fpath.poten_litera_db_kw_count
# output_path = fpath.poten_litera_db_ranked
# rank(input_path, output_path, params.ranking_params_weights)
# ---------------------end of test code---------------------

In [ ]:
def rank_and_save(input_path):
    # sort
    df_to_rank = pd.read_csv(input_path, header=0, sep="\t")

    df_to_rank.sort_values(by=['RELEVANCE_INDEX'], inplace=True, ascending=False)
        
    df_to_rank.reset_index(drop=True, inplace=True)
    
    df_to_rank.to_csv(input_path, header=True, index=False, sep="\t")

## Main program:

### 1. Calculate relevance index and rank

In [ ]:
rank_by = 'tak'
rank_by = '500'
rank_by = 'full_text'

In [ ]:
input_path = fpath.poten_litera_db_kw_count
relevance_index_path = fpath.poten_litera_db_relevance_index
# clear file
plib.clear_file(relevance_index_path)

compute_relevance_index(input_path, relevance_index_path, rank_by)

if rank_by == 'tak':
    # rank by tak    
    rank_and_save(fpath.poten_litera_db_ranked_by_tak)
elif rank_by == '500':
    # rank by 500    
    rank_and_save(fpath.poten_litera_db_ranked_by_500)
elif rank_by == 'full_text':
    # rank by full text
    rank_and_save(fpath.poten_litera_db_ranked_by_full_text)

### 3. Ranking results analysis

In [ ]:
# read the ranked database and obtain the relevance_index of YESs and NOs 
test_path = fpath.poten_litera_testing_set_1000_labeled
df_test = pd.read_csv(test_path, header=0, sep=',')

In [ ]:
# save the counts of the keywords in the respective lists
relevant_species = []
relevant_other_species = []
relevant_tc_ct = []
relevant_thalam = []
relevant_cortex = []
relevant_inject = []
relevant_method = []
relevant_connectivity = []

non_relevant_species = []
non_relevant_other_species = []
non_relevant_tc_ct = []
non_relevant_thalam = []
non_relevant_cortex = []
non_relevant_inject = []
non_relevant_method = []
non_relevant_connectivity = []

relvant_index_list = []
relevant_relevance_index_list = []

non_relevant_index_list = []
non_relevant_relevance_index_list = []

In [ ]:
if rank_by == 'tak':
    # rank by tak
    df_db_ranked = pd.read_csv(fpath.poten_litera_db_ranked_by_tak, header=0, sep='\t')
    column_name_base = "TAK"
elif rank_by == '500':
    # rank by 500
    df_db_ranked = pd.read_csv(fpath.poten_litera_db_ranked_by_500, header=0, sep='\t')
    column_name_base = "500"
elif rank_by == 'full_text':
    # rank by full text
    df_db_ranked = pd.read_csv(fpath.poten_litera_db_ranked_by_full_text, header=0, sep='\t')
    column_name_base = "FULL_TEXT"

for ind in df_test.index:
    index = int(df_test.at[ind, "INDEX"])

    if df_test.at[ind, "RELEVANT?(Y/N/MB/NA)"] == "Y":
        relvant_index_list.append(index)
        relevant_relevance_index_list.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "RELEVANCE_INDEX"].values[0])
        relevant_species.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "MACAQUE_COUNT_IN_"+column_name_base].values[0])
        relevant_other_species.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "OTHER_SPIECIES_COUNT_IN_"+column_name_base].values[0])
        relevant_tc_ct.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "TC_CT_COUNT_IN_"+column_name_base].values[0])
        relevant_thalam.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "THALAM_COUNT_IN_"+column_name_base].values[0])
        relevant_cortex.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "CORTEX_COUNT_IN_"+column_name_base].values[0])
        relevant_inject.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "INJECT_COUNT_IN_"+column_name_base].values[0])
        relevant_method.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "METHOD_COUNT_IN_"+column_name_base].values[0])
        relevant_connectivity.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "CONNECT_COUNT_IN_"+column_name_base].values[0])
    # elif df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "LENGTH_TEXT"].values[0] > 100:
    else:
        non_relevant_index_list.append(index)
        non_relevant_relevance_index_list.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "RELEVANCE_INDEX"].values[0])
        non_relevant_species.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "MACAQUE_COUNT_IN_"+column_name_base].values[0])
        non_relevant_other_species.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "OTHER_SPIECIES_COUNT_IN_"+column_name_base].values[0])
        non_relevant_tc_ct.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "TC_CT_COUNT_IN_"+column_name_base].values[0])
        non_relevant_thalam.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "THALAM_COUNT_IN_"+column_name_base].values[0])
        non_relevant_cortex.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "CORTEX_COUNT_IN_"+column_name_base].values[0])
        non_relevant_inject.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "INJECT_COUNT_IN_"+column_name_base].values[0])
        non_relevant_method.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "METHOD_COUNT_IN_"+column_name_base].values[0])
        non_relevant_connectivity.append(df_db_ranked.loc[df_db_ranked["INDEX"].apply(int) == index, "CONNECT_COUNT_IN_"+column_name_base].values[0])

In [ ]:
# plot the 8 dot plots of the species_related, tc_ct_related, thalam_related, cortex_related, method_related, connectivity_related of YESs and NOs of the test data set in 2 rows in the same figure
plt.figure(figsize=(10, 10))

plt.subplot(4, 2, 1)
plt.plot(relvant_index_list, relevant_species, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_species, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Species Related")
plt.legend()

plt.subplot(4, 2, 2)
plt.plot(relvant_index_list, relevant_other_species, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_other_species, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Other Species Related")
plt.legend()

plt.subplot(4, 2, 3)
plt.plot(relvant_index_list, relevant_tc_ct, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_tc_ct, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("TC_CT Related")
plt.legend()

plt.subplot(4, 2, 4)
plt.plot(relvant_index_list, relevant_thalam, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_thalam, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Thalam Related")
plt.legend()

plt.subplot(4, 2, 5)
plt.plot(relvant_index_list, relevant_cortex, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_cortex, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Cortex Related")
plt.legend()

plt.subplot(4, 2, 6)
plt.plot(relvant_index_list, relevant_cortex, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_cortex, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Inject Related")
plt.legend()

plt.subplot(4, 2, 7)
plt.plot(relvant_index_list, relevant_method, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_method, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Method Related")
plt.legend()

plt.subplot(4, 2, 8)
plt.plot(relvant_index_list, relevant_connectivity, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_connectivity, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Connectivity Related")
plt.legend()

plt.show()

In [ ]:
# plot the dot plot of the relevance_index of YESs and NOs of the test data set
plt.figure(figsize=(10, 5))
plt.plot(relvant_index_list, relevant_relevance_index_list, 'ro', label="YES")
plt.plot(non_relevant_index_list, non_relevant_relevance_index_list, 'bo', label="NO")
plt.xlabel("Index")
plt.ylabel("Relevance Index")
plt.legend()

# add labels for the relevant index points
for i, index in enumerate(relvant_index_list):
    plt.text(index, relevant_relevance_index_list[i]+0.1, str(index), color='black', fontsize=10)

# # add labels for the relevant index points
# for i, index in enumerate(non_relevant_index_list):
#     plt.text(index, non_relevant_relevance_index_list[i], str(index), color='black', fontsize=10)

plt.show()

In [ ]:
min_relevance_index_index = 3683

In [ ]:
# for i in range(len(relvant_index)):
    # print(relvant_index[i], relevant_relevance_index_list[i])
    # print(relevant_relevance_index_list)
# min_relev_index = min(relevant_relevance_index_list)

for ind in df_db_ranked.index:
    if df_db_ranked.at[ind, "INDEX"] == min_relevance_index_index:
        min_relevance_index_ind = ind
        break 
# print(min_relev_index_ind)

min_relev_index_num = min_relevance_index_ind  + 1
num = math.ceil(min_relev_index_num * 1.3)

threshold_index = df_db_ranked.at[num-1, 'INDEX']
print(f"The index with the minimum relevance index: {min_relevance_index_index}")
print(f"The number of articles with relevance index larger or equal than {min_relevance_index_index}: {min_relev_index_num}")
print(f"The index of the article with the threshold relevance index: {threshold_index}")
print(f"The number of articles to manually check is {num}")

In [ ]:
def pick_values_uniformly(data, n):
    """Pick up `n` values uniformly from `data`."""
    if n <= 0:
        return []

    # Determine the range of the data
    min_val, max_val = min(data), max(data)

    threshold = (max_val - min_val) / n / 2

    # If n is 1, just return the midpoint
    if n == 1:
        return [(min_val + max_val) / 2]

    # Calculate the interval size
    interval = (max_val - min_val) / (n - 1)

    # Get the uniform values
    return [min_val + i * interval for i in range(n)], threshold

# data = [1, 3, 5, 2, 8, 10, 2]
n = 5
density_display_index, thres = pick_values_uniformly(relevant_relevance_index_list + non_relevant_relevance_index_list, n)
print(density_display_index)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Printing the length of lists
print("Numer of relevant literature:", len(relevant_relevance_index_list))
print("Number of not relevant literature:", len(non_relevant_relevance_index_list))
print()

# Create a DataFrame for plotting
df = pd.DataFrame({'Relevance Index': relevant_relevance_index_list + non_relevant_relevance_index_list, 
                   'Label': ['Relevant'] * len(relevant_relevance_index_list) + ['Not Relevant'] * len(non_relevant_relevance_index_list)})

# Draw the violin plot
plt.figure(figsize=(10, 6))
ax = sns.violinplot(x='Label', y='Relevance Index', data=df, bw='scott', cut=0)

relevance_indices = density_display_index  # Replace with your relevance indices

threshold = thres  # Adjust this based on your desired range around the relevance index

for index in relevance_indices:
    ax.axhline(index, color='gray', linestyle='--')
    
    for i, label in enumerate(df['Label'].unique()):
        # Filter data points close to the current relevance index
        close_points = df[(df['Label'] == label) & (np.abs(df['Relevance Index'] - index) < threshold)]
        density = len(close_points)
        
        ax.text(i, index + 0.1, str(density), ha='center', va='center', color='red', fontsize=9)  # adjust the vertical offset (0.1 here) as necessary

plt.title('Distribution of Relevance Index')
plt.show()

In [ ]:
import scipy.stats as stats

# Check the equality of variances
var_relevant = np.var(relevant_relevance_index_list)
var_non_relevant = np.var(non_relevant_relevance_index_list)
print('Variance of relevant:', var_relevant)
print('Variance of non-relevant:', var_non_relevant)
print(var_relevant/var_non_relevant)
# statistic, p_value = stats.levene(relevant_relevance_index_list, non_relevant_relevance_index_list)

# # Print the results
# print('Levene test statistic:', statistic)
# print('p-value:', p_value)

<h3> Next step: manually read papers and find all actually related literature </h3>